In [44]:
import networkx as nx

In [45]:
import tqdm
import pandas as pd
import numpy as np

# Prune

In the previous notebook we saw that our pruning method affected negatively the clustering, as the result of our pruning was a very centralized network. One of the interesting things about this network is precisely the clustering. In this notebook we will try to figure out a way to preserve this different clusters. So instead of focusing on centrality, we focus on heterogeniety

## Remove nodes that ressemble other nodes

In [3]:
p_projection = nx.read_gpickle('../data/playlists_projection.gpkl')

### Filtering edges with weight less than

In [4]:
weight = nx.get_edge_attributes(p_projection, 'weight')
len([p_projection.remove_edge(k[0],k[1]) for k,v in weight.items() if v<15])

1902833

In [5]:
# np.unique([len(c) for c in nx.connected_components(p_projection)])

In [6]:
filtered_g = p_projection.subgraph(list(nx.connected_components(p_projection))[0]).copy()

In [7]:
# [c for c in nx.connected_components(p_projection) if len(c)==68]
filtered_g

In [8]:
len(filtered_g.nodes()),len(filtered_g.edges())

(3150, 41244)

In [9]:
len(filtered_g.edges())

41244

### Filtering redundant nodes

In [10]:
# nx.get_edge_attributes(p_projection, 'weight')

In [11]:
# list(filtered_g.neighbors('37i9dQZF1DX2taNm7KfjOX'))

In [12]:
dict_n = {}
for i in tqdm.tqdm(filtered_g.nodes()):
    dict_n[i]= set(filtered_g.neighbors(i))

100%|██████████████████████████████████████████████████████████████████████████| 3150/3150 [00:00<00:00, 137068.76it/s]


In [13]:
matrix = pd.DataFrame(index=dict_n.keys(), columns=dict_n.keys())

In [14]:
import numpy as np

In [15]:
for k_1, v_1 in tqdm.tqdm(dict_n.items()):
    for k_2 in v_1:
        v_2 = dict_n[k_2]        
        matrix[k_1][k_2] = len(v_1.intersection(v_2)) / len(v_1.union(v_2))

100%|█████████████████████████████████████████████████████████████████████████████| 3150/3150 [00:09<00:00, 328.11it/s]


In [16]:
%matplotlib inline

In [17]:
mean_matrix = matrix.mean()

In [43]:
# [mean_matrix[mean_matrix>0.5]]

In [34]:
i=0
for k in mean_matrix[mean_matrix>0.2].index:
    if k in filtered_g:
        filtered_g.remove_node(k)
        i+=1
#     for v in dict_n[k]:
#         if v in p_projection:
#             p_projection.remove_node(v)
#             i+=1
i

673

In [35]:
len(filtered_g_2.nodes()),len(filtered_g_2.edges())

(1527, 3460)

In [36]:
arg_max = np.argmax([len(c) for c in nx.connected_components(filtered_g)])
arg_max

0

In [37]:
filtered_g_2 = filtered_g.subgraph(list(nx.connected_components(filtered_g))[arg_max])

In [38]:
len(filtered_g_2.nodes()),len(filtered_g_2.edges())

(1257, 3310)

In [42]:
# filtered_g_2.nodes()

In [48]:
nx.write_gpickle(filtered_g_2, 'playlists_projection_1257.pkl')

# Random graphs

In [ ]:
import itertools
import random
import matplotlib.pyplot as plt

%matplotlib inline
import networkx as nx